In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# Define arguments
input_size = 28 # Total size of image: 28 * 28
num_classes = 10 # Number of label types
num_epochs = 3 # Total training cycles
batch_size = 64 # Size of a batch, 64 images

# Training set
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

# Testing set
test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

# Construct batch data
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(     # Input size (1, 28, 28)
            nn.Conv2d(
                in_channels=1,          # Gray image  
                out_channels=16,        # How many feature images should be obtained
                kernel_size=5,          # Convolution kernel size
                stride=1,               # Stride size
                padding=2               # If the size of convolution is expected to be the same as the origin, this should be set: padding=(kernel_size-1)/2 if stride=1
            ),                          # The output feature map is (16, 28, 28)
            nn.ReLU(),                  # ReLU layer
            nn.MaxPool2d(kernel_size=2) # Do pooling operations (2x2 area), the output is: (16, 14, 14)
        )
        self.conv2 = nn.Sequential(     # The following input is (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2), # Output: (32, 14, 14)
            nn.ReLU(),                  # ReLU layer
            nn.MaxPool2d(kernel_size=2) # Output: (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)    # Result obtained from the full-linked layer
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       # Flatten operation, output is: (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

In [ ]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [ ]:
# Instantiate
net = CNN()
# The loss function
criterion = nn.CrossEntropyLoss()
# The optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001) # Define the optimizer, the normal random gradient downward algorithm

# Start the training loop
for epoch in range(num_epochs):
    # Store the result of the current epoch
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader): # Do loop for every batch within the container
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
                
            # Accuracy calculation
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum(tup[1] for tup in val_rights))
            
            print('Current epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy of the training set: {:.2f}%\tAccuracy of the testing set: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0].numpy() / train_r[1],
                100. * val_r[0].numpy() / val_r[1]
            ))